In [1]:
import os
from keras.preprocessing import image
import cv2

In [2]:
categories=['with_mask','without_mask']

In [3]:
data=[]
for category in categories:
    path=os.path.join('data',category)
    label=categories.index(category)
    
    for file in os.listdir(path):
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])

In [4]:
import random
random.shuffle(data)

In [5]:
X=[]
y=[]
for features,label in data:
    X.append(features)
    y.append(label)

In [6]:
import numpy as np
X=np.array(X)
y=np.array(y)

In [7]:
X=X/255

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [9]:
from keras.applications.vgg16 import VGG16

In [10]:
vgg=VGG16()

In [11]:
from keras import Sequential

In [12]:
model=Sequential()
for layer in vgg.layers[:-1]:
    model.add(layer)

In [13]:
for layer in model.layers:
    layer.trainable=False  

In [14]:
from keras.layers import Dense

In [15]:
model.add(Dense(1,activation='sigmoid'))

In [16]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [17]:
from tensorflow.keras import layers

In [18]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))


Epoch 1/5
35/35 [==============================] - 680s 19s/step - loss: 0.6240 - accuracy: 0.6836 - val_loss: 0.4934 - val_accuracy: 0.7790
Epoch 2/5
35/35 [==============================] - 611s 18s/step - loss: 0.3672 - accuracy: 0.9009 - val_loss: 0.3302 - val_accuracy: 0.8877
Epoch 3/5
35/35 [==============================] - 596s 17s/step - loss: 0.2700 - accuracy: 0.9227 - val_loss: 0.2447 - val_accuracy: 0.9529
Epoch 4/5
35/35 [==============================] - 640s 18s/step - loss: 0.2244 - accuracy: 0.9400 - val_loss: 0.2037 - val_accuracy: 0.9457
Epoch 5/5
35/35 [==============================] - 608s 17s/step - loss: 0.1925 - accuracy: 0.9509 - val_loss: 0.1783 - val_accuracy: 0.9529


In [19]:
cap=cv2.VideoCapture(0)


In [32]:
def detect_face_mask(img):
    y_pred=model.predict(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [37]:
def draw_label(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][1]-2
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [39]:

while True:
    ret,frame=cap.read()
    #call detection method
    img=cv2.resize(frame,(224,224))
    y_pred=detect_face_mask(img)
    if y_pred==0:
        draw_label(frame,"Mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"Face Mask Detection",(30,30),(255,0,0))
    print(y_pred)
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF==ord('x'):
        break

cv2.destroyAllWindows()


1/1 [==============================] - 0s 225ms/step
0.0076051927
1/1 [==============================] - 0s 244ms/step
0.049537893
1/1 [==============================] - 0s 254ms/step
0.014611186
1/1 [==============================] - 0s 266ms/step
0.017330416
1/1 [==============================] - 0s 279ms/step
0.016084617
1/1 [==============================] - 0s 271ms/step
0.02510827
1/1 [==============================] - 0s 263ms/step
0.014294917
1/1 [==============================] - 0s 273ms/step
0.0085814325
1/1 [==============================] - 0s 274ms/step
0.031512707
1/1 [==============================] - 0s 256ms/step
0.0110017145
1/1 [==============================] - 0s 289ms/step
0.024411628
1/1 [==============================] - 0s 282ms/step
0.07100899


KeyboardInterrupt: 